### Name: Buraq Khan

### Email: i180800@nu.edu.pk

### Roll num: i180800

In [81]:
import re
import os
import math

*Expand each heading to see working*

# File Reading

In [82]:
fake_doc = ""
count_fake = 0 
for filename in os.listdir('data/Train/Fake'):
    count_fake += 1
    with open(os.path.join('data/Train/Fake', filename), 'r') as f:
        fake_doc += f.read()
        f.close()

real_doc = ""
count_real = 0 
for filename in os.listdir('data/Train/Real'):
    count_real += 1
    with open(os.path.join('data/Train/Real', filename), 'r') as f:
        real_doc += f.read()
        f.close()


In [83]:
fake_doc = re.sub('\s+', ' ', fake_doc)
real_doc = re.sub('\s+', ' ', real_doc)
#count_fake
count_real

350

In [84]:
f = open("data/stopwords-ur.txt", "r")
stop_words = f.readlines()

for count, word in enumerate(stop_words):
    word = re.sub("\s+", "", word)
    stop_words[count] = word

## Duplicate Working

In [85]:
dup_fake_list = []
for filename in os.listdir('data/Train/Fake'):
    with open(os.path.join('data/Train/Fake', filename), 'r') as f:
        dup_fake_list.append(f.read())
        f.close()

dup_real_list = []
for filename in os.listdir('data/Train/Real'):
    with open(os.path.join('data/Train/Real', filename), 'r') as f:
        dup_real_list.append(f.read())
        f.close()

for count, text in enumerate(dup_fake_list):
    text = re.sub('\s+', ' ', text)
    dup_fake_list[count] = text

for count, text in enumerate(dup_real_list):
    text = re.sub('\s+', ' ', text)
    dup_real_list[count] = text

In [86]:
#removing duplicates
dup_fake = ""
for text in dup_fake_list:
    l = text.split()
    k = []
    for word in l:
        if (text.count(word)>1 and (word not in k) or text.count(word)==1):
            k.append(word) 
    dup_fake += ' '.join(k)

In [87]:
dup_real = ""
for text in dup_real_list:
    l = text.split()
    k = []
    for word in l:
        if (text.count(word)>1 and (word not in k) or text.count(word)==1):
            k.append(word) 
    dup_real += ' '.join(k)

# Learning through Naive Bayes (Training)

## Calculating Priors

In [88]:
fake_prior = count_fake / (count_fake + count_real)
real_prior = count_real / (count_fake + count_real)
print(fake_prior, real_prior)

0.45141065830721006 0.54858934169279


## Finding Word occurences

### With Stop words

In [89]:
# Calculating the occurence of each word in fake doc
fake_occur = {}
l = fake_doc.split()
for word in l:
    if word not in fake_occur.keys():
        fake_occur[word] = 1
    else:
        fake_occur[word] += 1


In [91]:
real_occur = {}
l = real_doc.split()
for word in l:
    if word not in real_occur.keys():
        real_occur[word] = 1
    else:
        real_occur[word] += 1


In [93]:
vocabulary = []
text = real_doc + fake_doc
text_list = text.split()
for word in text_list:
    if word not in vocabulary:
        vocabulary.append(word)

In [94]:
len(vocabulary)

16036

### Without Stop words

In [99]:
# Calculating the occurence of each word in fake doc
fake_occur_sw = {}
l = fake_doc.split()
for word in l:
    if word not in stop_words:
        if word not in fake_occur_sw.keys():
            fake_occur_sw[word] = 1
        else:
            fake_occur_sw[word] += 1


In [100]:
real_occur_sw = {}
l = real_doc.split()
for word in l:
    if word not in stop_words:
        if word not in real_occur_sw.keys():
            real_occur_sw[word] = 1
        else:
            real_occur_sw[word] += 1


### Removing Duplicates

In [102]:
dup_fake_dict = {}
l = dup_fake.split()
for word in l:
    if word not in dup_fake_dict.keys():
        dup_fake_dict[word] = 1
    else:
        dup_fake_dict[word] += 1

# for key, val in dup_fake_dict.items():
#     print(key, val)

In [103]:
dup_real_dict = {}
l = dup_real.split()
for word in l:
    if word not in dup_real_dict.keys():
        dup_real_dict[word] = 1
    else:
        dup_real_dict[word] += 1

# for key, val in dup_real_dict.items():
#     print(key, val)

### Removing Negation Words

In [119]:
negation_words = ['نہیں', 'نہ']
fake_dict_neg = {}
real_dict_neg = {}
for key, val in fake_occur.items():
    if key not in negation_words:
        fake_dict_neg[key] = val
for key, val in real_occur.items():
    if key not in negation_words:
        real_dict_neg[key] = val

## Conditional Probabilities (using Laplace Smoothing)

In [95]:
def calcNBFake(fake_dict, vocab):
    prob_fake = {}
    for word in vocab:
        if word in fake_dict.keys():
            prob_fake[word] = (fake_dict[word] + 1) / (len(fake_dict.keys()) + len(vocab))  # laplace smoothing
        else:
            prob_fake[word] = 1 / (len(fake_dict.keys()) + len(vocab))
    return prob_fake


In [97]:
def calcNBReal(real_dict, vocab):
    prob_real = {}
    for word in vocab:
        if word in real_dict.keys():
            prob_real[word] = (real_dict[word] + 1) / (len(real_dict.keys()) + len(vocab))
        else:
            prob_real[word] = 1 / (len(real_dict.keys()) + len(vocab))

    return prob_real


In [96]:
fake_prob = calcNBFake(fake_occur, vocabulary)
real_prob = calcNBReal(real_occur, vocabulary)

In [101]:
fake_prob_sw = calcNBFake(fake_occur_sw, vocabulary)
real_prob_sw = calcNBReal(real_occur_sw, vocabulary)

In [104]:
dup_real_prob = calcNBReal(dup_real_dict, vocabulary)
dup_fake_prob = calcNBFake(dup_fake_dict, vocabulary)

In [120]:
fake_prob_neg = calcNBFake(fake_dict_neg, vocabulary)
real_prob_neg = calcNBReal(real_dict_neg, vocabulary)

# Naive Bayes (Testing)

In [105]:
def NBTest(real_prior, fake_prior, real_prob, fake_prob, test_words):
    fake_score = math.log(fake_prior) + 10
    real_score = math.log(real_prior)
    for word in test_words:
        if word in vocabulary:
            fake_score += math.log(fake_prob[word])
            real_score += math.log(real_prob[word])
    
    if fake_score > real_score:
        return "Fake"
    else:
        return "Real"

### Testing output

In [106]:
f = open('data/Test/Fake/tch71.txt', "r")
test = f.read()
idk = test.split()
words = []
words_sw = []
for word in idk:
    if word not in words:
        words.append(word)
    if word not in words_sw and word not in stop_words:
        words_sw.append(word)


In [107]:
NBTest(real_prior, fake_prior, real_prob, fake_prob, words)
#NBTest(real_prior, fake_prior, real_prob_sw, fake_prob_sw, words_sw)
#NBTest(real_prior, fake_prior, dup_real_prob, dup_fake_prob, words)


'Real'

# Evaluation

## Real

In [108]:
def calcReal(r_prior, f_prior, r_prob, f_prob):
    total_real = 0
    true_real = 0 
    false_fake = 0
    for filename in os.listdir('data/Test/Real'):
        total_real += 1
        with open(os.path.join('data/Test/Real', filename), 'r') as f:
            text = f.read()
            f.close()
        
        text_words = text.split()
        result = NBTest(r_prior, f_prior, r_prob, f_prob, text_words)
        if result == "Real":
            true_real += 1
        else:
            false_fake += 1
    
    return total_real, true_real, false_fake

## Fake

In [109]:
def calcFake(r_prior, f_prior, r_prob, f_prob):
    total_fake = 0
    true_fake = 0 
    false_real = 0
    for filename in os.listdir('data/Test/Fake'):
        total_fake += 1
        with open(os.path.join('data/Test/Fake', filename), 'r') as f:
            text = f.read()
            f.close()
        
        text_words = text.split()
        result = NBTest(r_prior, f_prior, r_prob, f_prob, text_words)
        if result == "Fake":
            true_fake += 1
        else:
            false_real += 1
    
    return total_fake, true_fake, false_real

## Evaluation Metrics

In [110]:
def Accuracy(tr, tf, fr, ff):
    return (tr + tf) / (tr + tf + fr + ff)

In [111]:
def Precision(tp, fp):
    return tp / (tp + fp)

p = Precision(true_real, false_real)

In [112]:
def Recall(tp, fn):
    return tp / (fn + tp)

r = Recall(true_real, false_fake)

In [113]:
def F1(precision, recall):
    return (2 * precision * recall) / (precision + recall)

In [114]:
def Testing(r_prob, f_prob):
    total_real, true_real, false_fake = calcReal(real_prior, fake_prior, r_prob, f_prob)
    total_fake, true_fake, false_real = calcFake(real_prior, fake_prior, r_prob, f_prob)
    
    accuracy = Accuracy(true_real, true_fake, false_real, false_fake)
    
    precision_real = Precision(true_real, false_real)
    precision_fake = Precision(true_fake, false_fake)
    
    recall_real = Recall(true_real, false_fake)
    recall_fake = Recall(true_fake, false_real)

    F1_real = F1(precision_real, recall_real)
    F1_fake = F1(precision_fake, recall_fake)

    return accuracy, precision_real, precision_fake, recall_real, recall_fake, F1_real, F1_fake

In [115]:
def output(r_prob, f_prob):
    accuracy, precision_real, precision_fake, recall_real, recall_fake, F1_real, F1_fake = Testing(r_prob, f_prob)
    print("Accuracy: ", accuracy)
    print("Precision of real class: ", precision_real)
    print("Precision of fake class: ", precision_fake)
    print("Recall of real class: ", recall_real)
    print("Recall of fake class: ", recall_fake)
    print("F1 of real class: ", F1_real)
    print("F1 of fake class: ", F1_fake)


## With Stop Words 

In [116]:
print("Evaluation of Text Classifier with stop words included")
output(real_prob, fake_prob)

Evaluation of Text Classifier with stop words included
Accuracy:  0.5954198473282443
Precision of real class:  0.5859375
Precision of fake class:  1.0
Recall of real class:  1.0
Recall of fake class:  0.05357142857142857
F1 of real class:  0.7389162561576355
F1 of fake class:  0.10169491525423728


## Without Stop Words

In [117]:
print("Evaluation of Text Classifier without stop words included")
output(real_prob_sw, fake_prob_sw)

Evaluation of Text Classifier without stop words included
Accuracy:  0.6679389312977099
Precision of real class:  0.6375545851528385
Precision of fake class:  0.8787878787878788
Recall of real class:  0.9733333333333334
Recall of fake class:  0.25892857142857145
F1 of real class:  0.770448548812665
F1 of fake class:  0.4


## Boolean Naive Bayes

In [118]:
print("Evaluation of Text Classifier using Boolean Naive Bayes")
output(dup_real_prob, dup_fake_prob)

Evaluation of Text Classifier using Boolean Naive Bayes
Accuracy:  0.6564885496183206
Precision of real class:  0.6293103448275862
Precision of fake class:  0.8666666666666667
Recall of real class:  0.9733333333333334
Recall of fake class:  0.23214285714285715
F1 of real class:  0.7643979057591623
F1 of fake class:  0.3661971830985915


## After Removing Negation Words (Bonus)

In [121]:
print("Evaluation of Text Classifier after removing Negation Words")
output(real_prob_neg, fake_prob_neg)

Evaluation of Text Classifier after removing Negation Words
Accuracy:  0.5954198473282443
Precision of real class:  0.5859375
Precision of fake class:  1.0
Recall of real class:  1.0
Recall of fake class:  0.05357142857142857
F1 of real class:  0.7389162561576355
F1 of fake class:  0.10169491525423728


# Report 

### **Comparing Naive Bayes with and without stopwords:**

### When we didn't remove stop words from our text, the average accuracy was roughly '59.5%'. Whereas, when we removed the stop words from our text, the average accuracy came to be '66.7%'. As seen clearly, the removal of stop words had an effect on the accuracy.

### Furthermore, without the stop words included, there is a significant improvement in the detection of fake news, both *Recall* and *F1* show a large improvement.

### From this we can deduce that on average the removal of stop words resulted in sizeable improvement in the correctness of our text classifier in the detection of Fake News.

### **Comparing Normal Naive Bayes with Boolean Naive Bayes Classifier:**

### Boolean Naive Bayes Classifier performs better at detecting Fake News as compared to Normal Naive Bayes Classifier. This can be seen in the results of Accuracy, Recall and F1. 

### **Comparing Naive Bayes after removal of Negation words:**

### There is very little improvement in the detection of Fake News after removal of Negation Words in comparision with normal data. Although, it seems like if certain phrases that indicated negation were removed in preprocessing it would improve results. 